Link to Kaggle dataset: https://www.kaggle.com/datasets/deathtrooper/multichannel-glaucoma-benchmark-dataset/data

In [1]:
import zipfile
import pandas as pd
import keras
import os
import numpy as np
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Input, Concatenate, ReLU, AveragePooling2D, UpSampling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 
from keras.layers import GlobalMaxPool2D

In [2]:
# Unzip files for processing
with zipfile.ZipFile("full-fundus.zip","r") as zip_ref:
    zip_ref.extractall("full-fundus")
    
with zipfile.ZipFile("blood-vessel.zip","r") as zip_ref:
    zip_ref.extractall("blood-vessel")


In [3]:
# Read in metadata, and filter for a specific subset of images
# TODO: will add more variety of images later
image_data = pd.read_csv('metadata.csv')
image_data = image_data[image_data['names'].str.contains('FIVES', case=True) | image_data['names'].str.contains('HAGIS', case=True)
                       | image_data['names'].str.contains('LES-AV', case=True) | image_data['names'].str.contains('G1020', case=True)]
#g1020 = image_data[image_data['names'].str.contains('G1020', case=True)]
#g1020 = g1020[g1020["fundus_oc_seg"] != "Not Visible"]

#image_data = pd.concat([subset_image_data, g1020])
image_data = image_data[['types', 'fundus', 'names']]

In [4]:
# There are some invalid data types for relevant columns
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1452 entries, 4524 to 12448
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   types   1452 non-null   int64 
 1   fundus  1452 non-null   object
 2   names   1452 non-null   object
dtypes: int64(1), object(2)
memory usage: 45.4+ KB
None


In [5]:
# Change datatypes to the desired values
image_data['types'] = image_data['types'].astype('string')
image_data['fundus'] = image_data['fundus'].astype('string')
image_data['names'] = image_data['names'].astype('string')
image_data['image_names'] = image_data['names'] + '.png'


In [6]:
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1452 entries, 4524 to 12448
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   types        1452 non-null   string
 1   fundus       1452 non-null   string
 2   names        1452 non-null   string
 3   image_names  1452 non-null   string
dtypes: string(4)
memory usage: 56.7 KB
None


In [7]:
# Randomize data 
image_data_random = image_data.sample(frac=1, random_state=1)

# Split into healthy and glaucoma positive sets
healthy = image_data_random[image_data_random['types'] == "0"]
glaucoma = image_data_random[image_data_random['types'] == "1"]

print(f"Number of healthy samples: {len(healthy)}")
print(f"Number of unhealthy samples: {len(glaucoma)}")

Number of healthy samples: 985
Number of unhealthy samples: 467


In [8]:
# Partition data into test and train sets
healthy_train_size = 689
glaucoma_train_size = 326

healthy_test_subset = image_data_random.head(len(healthy) - healthy_train_size)
glaucoma_test_subset = image_data_random.head(len(glaucoma) - glaucoma_train_size)
test_subset = pd.concat([healthy_test_subset, glaucoma_test_subset])

healthy_train_subset = image_data_random.tail(healthy_train_size)
glaucoma_train_subset = image_data_random.tail(glaucoma_train_size)
train_subset = pd.concat([healthy_train_subset, glaucoma_train_subset])

print(f"Test size: {len(test_subset)}")
print(f"Train size: {len(train_subset)}")

Test size: 437
Train size: 1015


In [9]:
train_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_train_data = train_data_generator.flow_from_dataframe(dataframe=train_subset, 
                                            batch_size= 8, 
                                            shuffle=True, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')

test_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_test_data = train_data_generator.flow_from_dataframe(dataframe=test_subset, 
                                            batch_size= 1, 
                                            shuffle=False, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')



Found 1015 validated image filenames belonging to 2 classes.
Found 437 validated image filenames belonging to 2 classes.


In [10]:
def conv_model(image_size):
    
    m_input = keras.Input(shape=(image_size, image_size, 3))
    transfer = keras.applications.MobileNetV3Large(
        weights='imagenet', include_top= False, input_tensor= m_input, alpha=0.75
    )
    m_output = Dropout(0.5)(transfer.output)
    m_output = Conv2D(filters=256, kernel_size=1)(m_output) 
    m_output = GlobalMaxPool2D()(m_output)
    m_output = Dropout(0.5)(m_output)
    m_output = Dense(2, activation='softmax')(m_output)

    return keras.Model(inputs=m_input, outputs=m_output)

model = conv_model(image_size=224)

# Reduce the learning rate if an epoch occurs where there is no improvement to the output of the loss function
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=1, min_lr=1e-6)

# Using the Adam optimizer with binary cross entropy, compile the model using the given metrics
model.compile(optimizer= Adam(1e-5), 
              loss='binary_crossentropy', 
              metrics=['binary_accuracy',keras.metrics.AUC(),keras.metrics.Precision(), keras.metrics.Recall()])

trained_model = model.fit(flow_train_data, 
                    steps_per_epoch= len(flow_train_data) // 8,
                    validation_data= flow_test_data, 
                    validation_steps= len(flow_test_data), 
                    epochs=20, 
                    callbacks=[reduce_lr])

Epoch 1/20


/Users/ThomasMcBride/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 515ms/step - auc: 0.5567 - binary_accuracy: 0.5446 - loss: 4.4908 - precision: 0.5446 - recall: 0.5446 - val_auc: 0.4181 - val_binary_accuracy: 0.4256 - val_loss: 2.2646 - val_precision: 0.4256 - val_recall: 0.4256 - learning_rate: 1.0000e-05
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - auc: 0.5271 - binary_accuracy: 0.5248 - loss: 5.1763 - precision: 0.5248 - recall: 0.5248 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05
Epoch 3/20


2024-03-29 22:07:48.424712: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/ThomasMcBride/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 490ms/step - auc: 0.5850 - binary_accuracy: 0.5533 - loss: 4.2939 - precision: 0.5533 - recall: 0.5533 - val_auc: 0.4610 - val_binary_accuracy: 0.4485 - val_loss: 1.9990 - val_precision: 0.4485 - val_recall: 0.4485 - learning_rate: 1.0000e-05
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - auc: 0.5354 - binary_accuracy: 0.5287 - loss: 4.7326 - precision: 0.5287 - recall: 0.5287 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05
Epoch 5/20


...

2024-03-29 22:09:00.096109: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 495ms/step - auc: 0.5475 - binary_accuracy: 0.5371 - loss: 3.8337 - precision: 0.5371 - recall: 0.5371 - val_auc: 0.5157 - val_binary_accuracy: 0.5195 - val_loss: 1.4894 - val_precision: 0.5195 - val_recall: 0.5195 - learning_rate: 1.0737e-06
Epoch 18/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - auc: 0.5490 - binary_accuracy: 0.5132 - loss: 3.3552 - precision: 0.5132 - recall: 0.5132 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-06
Epoch 19/20


2024-03-29 22:09:08.645071: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-29 22:09:08.651869: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 514ms/step - auc: 0.6223 - binary_accuracy: 0.6325 - loss: 3.9253 - precision: 0.6325 - recall: 0.6325 - val_auc: 0.5152 - val_binary_accuracy: 0.5126 - val_loss: 1.4891 - val_precision: 0.5126 - val_recall: 0.5126 - learning_rate: 1.0000e-06
Epoch 20/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 247ms/step - auc: 0.6696 - binary_accuracy: 0.6547 - loss: 2.8791 - precision: 0.6547 - recall: 0.6547 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-06


2024-03-29 22:09:20.057000: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [11]:
# Demonstration:
print(trained_model)
img_path = 'full-fundus/full-fundus/OIA-ODIR-TEST-ONLINE-252.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation)  # Assuming preprocess_input is defined in your code

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
else:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
[[0.9604122  0.03958783]]
Prediction: Glaucoma, with 96.04122042655945% confidence
